In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.optim import optimizer
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch.optim.lr_scheduler as lr_scheduler
from datetime import datetime, timedelta

## 定義 Regression 的類別

In [2]:
class LinearRegression(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.net = nn.Sequential (
            nn.Linear(input_dim,output_dim),
#             nn.ReLU(),
#             nn.Linear(input_dim,8),
#             nn.ReLU(),
#             nn.Linear(8,output_dim),
#             nn.ReLU(),
        )
    def forward(self, x):
        return self.net(x)


## Load Data

In [3]:
def load_data(folder_path):
    load_df = pd.DataFrame()
  # 遍歷文件夾中的所有文件
    name_list=[]
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            count = 0
            date_list = []
            for i in range(len(df["Date"])-1,-1, -1):
                date_string = df["Date"][i]
                dt = datetime.strptime(date_string, '%Y-%m-%d')
                date_list.append(dt)
                count = 1
            i = len(date_list) - 1  # 從列表的最後一個元素開始
            while i >= 0:
                if i == 0:
                    if count < 5:
                        for j in range(count):
                            del date_list[i-j]
                    break
                else:
                    if date_list[i] - timedelta(days=1) == date_list[i-1]:
                        count += 1
                    else:
                        if count < 5:
                            for j in range(count):
                                del date_list[i-j]
                            count = 1
                        else:
                            count = 1
                    i -= 1  # 向前移動 
            i = 0
            j = 0
            
            # 將 'Date' 欄位轉換為 datetime
            df['Date'] = pd.to_datetime(df['Date'])
            date_set = set(date_list)
            mask = df["Date"].isin(date_set)
            df = df[mask]
            df = df.sort_values(by="Date")
            df['Date'] = df['Date'].dt.strftime(('%Y-%m-%d'))
            df.set_index(df['Date'], inplace=True)

#             if filename == "AUD.csv":
#                 print (df)
#             load_df = pd.concat([load_df, df["Date"], df["現鈔買入"], df["現鈔賣出"], df["即期買入"], df["即期賣出"]], axis=1)
            
#             print(load_df)
            load_df = pd.concat([load_df, df["現鈔買入"]], axis=1)
#             load_df.fillna(0, inplace=True)
            filename = filename.replace(".csv", "")
            name_list.extend([f"{filename}現鈔買入"])
#             name_list.extend([f"{filename}Date", f"{filename}現鈔買入", f"{filename}現鈔賣出", f"{filename}即期買入", f"{filename}即期賣出"])
#     print(name_list)
#     print(load_df)
    load_df.columns = name_list
  # 反向排序
#     load_df = load_df.iloc[::-1]
    
  # 排序column
    load_df = load_df.reindex(sorted(load_df.columns), axis=1)

  # 處理空值
#     load_df.replace("-", 0, inplace=True)
#     load_df.fillna(0, inplace=True)
  #print(load_df)
    return load_df

#Kaggle 的路徑!!
# train_df = load_data("/kaggle/input/2023-artificial-intelligence-hw1/Dataset/train")



train_df = load_data("./Dataset/train")


# train_df['AUDDate'] = pd.to_datetime(train_df['AUDDate'])
# train_df = train_df.sort_values(by='AUDDate')
# train_df.replace("-", 0, inplace=True)
# train_df.replace("NaN",0, inplace=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.expand_frame_repr')
# train_df['AUDDate'] = train_df['AUDDate'].dt.strftime(('%Y-%m-%d'))

train_df.replace("-", 0, inplace=True)
train_df.fillna(0, inplace=True)

first_time = True
for i in range(len(train_df.columns)): #make the 0 be the half of the value numbers start and end
    for j in range(len(train_df)):
        start = 0
        if train_df.iloc[j,i] == 0:
            front = train_df.iloc[j-1,i]
            ti = i
            tj = j
            while(train_df.iloc[tj,ti] == 0):
                tj += 1
            end = train_df.iloc[tj,ti]
            if first_time:
                first_time = False
                print(front,end)
            half = (front + end)/2
            half = round(half, 2)
            train_df.iloc[j,i] = half
            
train_df.reset_index(drop=True, inplace=True)  # 重置索引，丢弃原有索引
print(train_df.shape)
train_df

IndexError: index 916 is out of bounds for axis 0 with size 916

## Preprocess
這邊的做法會以前四天的資料，來預測出第五天的結果

In [ ]:
# 設定輸入資料的天數範圍
input_date_data_size = 4

# 設定 seed
torch.manual_seed(66)
np.random.seed(66)

In [ ]:
train = train_df.to_numpy()
train_size, feature_size = train.shape
# 以一段時間的資料當作輸入，故資料數量要扣掉輸入天數範圍
train_size = train_size - input_date_data_size

train

In [ ]:
train_x = np.empty([train_size, feature_size * input_date_data_size], dtype = float)
train_y = np.empty([train_size, feature_size], dtype = float)

for idx in range(train_size):
  temp_data = np.array([])
  for count in range(input_date_data_size):
    temp_data = np.hstack([temp_data, train[idx + count]])
  train_x[idx, :] = temp_data
  train_y[idx, :] = train[idx + input_date_data_size]
# y值只留下現鈔買入
filtered_columns = [train_df.columns.get_loc(col) for col in train_df.columns if '現鈔買入'in col]
train_y = train_y[:, filtered_columns]
train_y


## Standardize

In [ ]:
mean_x = np.mean(train_x, axis = 0)
std_x = np.std(train_x, axis = 0)
for i in range(len(train_x)):
    for j in range(len(train_x[0])):
        if std_x[j] != 0:
            train_x[i][j] = (train_x[i][j] - mean_x[j]) / std_x[j]
            
# train_y = np.array(train_y)
            
# mean_y = np.mean(train_y, axis = 0)
# std_y = np.std(train_y, axis = 0)
# for i in range(len(train_y)):
#     for j in range(len(train_y[0])):
#         if std_y[j] != 0:
#             train_y[i][j] = (train_y[i][j] - mean_y[j]) / std_y[j]

# train_x, train_y

# train_x = np.nan_to_num(train_x, nan=0)

# min_val = np.min(train_x, axis=0)
# max_val = np.max(train_x, axis=0)
# print(train_x[0],min_val,max_val)

# for i in range(len(train_x)):
#     for j in range(len(train_x[0])):
#         if max_val[j] - min_val[j] != 0:
#             train_x[i][j] = (train_x[i][j] - min_val[j]) / (max_val[j] - min_val[j])
#         else:
#             train_x[i][j] = (train_x[i][j] - min_val[j])
# train_x[0],min_val,max_val

# row_sums = train_x.sum(axis=1)
# new_matrix = train_x / row_sums[:, np.newaxis]


## Split Validation

In [ ]:
split_ratio = 0.2

# 計算驗證集的大小
val_size = int(train_size * split_ratio)

# 隨機生成索引以選擇要包含在驗證集中的樣本
indices = np.random.permutation(train_size)

# 使用索引切分數據
val_indices = indices[:val_size]
train_indices = indices[val_size:]

# 創建訓練集和驗證集
val_x = train_x[val_indices]
val_y = train_y[val_indices]

train_x = train_x[train_indices]
train_y = train_y[train_indices]


## Training

In [ ]:
train_x = torch.from_numpy(train_x.astype(np.float32))
train_y = torch.from_numpy(train_y.astype(np.float32))
val_x = torch.from_numpy(val_x.astype(np.float32))
val_y = torch.from_numpy(val_y.astype(np.float32))
print(train_x[0],train_y[0],val_x[0],val_y[0])

In [ ]:
model = LinearRegression(feature_size * input_date_data_size, 8)

# learning_rate = 0.01
# criterion = nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.001, total_iters=10)

In [ ]:
epochs = 1000000

train_loss_history = []
val_loss_history = []

for epoch in range(epochs):
    if epoch == 10000:
        learning_rate = learning_rate*0.1
    if epoch == 100000:
        learning_rate = learning_rate*0.1
    model.train()
    # forward pass and loss
    train_y_predicted = model(train_x)
    loss = criterion(train_y_predicted, train_y)
    train_loss_history.append(loss.item())
    # backward pass
    loss.backward()
    # update
    optimizer.step()
    # init optimizer
    optimizer.zero_grad()
#     before_lr = optimizer.param_groups[0]["lr"]
#     scheduler.step()
#     after_lr = optimizer.param_groups[0]["lr"]
#     print("Epoch %d: SGD lr %.4f -> %.4f" % (epoch, before_lr, after_lr))


    # 驗證資料集
    model.eval()
    val_y_predicted = model(val_x)
    val_loss = criterion(val_y_predicted, val_y)
    val_loss_history.append(val_loss.item())
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, train_loss = {loss.item(): .4f}, val_loss = {val_loss.item(): .4f}')

## Chart drawing

In [ ]:
epochs_range = range(1, epochs + 1)
plt.plot(epochs_range, train_loss_history, 'ro', label='Train Loss')
plt.plot(epochs_range, val_loss_history, 'b', label='Validation Loss')

plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
train_predicted = model(train_x).detach().numpy()
val_predicted = model(val_x).detach().numpy()

plt.plot(train_y, train_predicted, 'ro')
plt.plot(val_y, val_predicted, 'b')
plt.show()

## Testing
test 資料集需要注意的事情是，我們會以每四筆輸入輸出一組預測結果。<br>
也就是 test 資料共有 3984 筆資料，因此我們會預測出 996 筆結果。

In [ ]:
def load_test_data(folder_path):
  load_df = pd.DataFrame()
  # 遍歷文件夾中的所有文件
  name_list=[]
  for filename in os.listdir(folder_path):
      if filename.endswith(".csv"):
          file_path = os.path.join(folder_path, filename)
          df = pd.read_csv(file_path)
#           load_df = pd.concat([load_df, df["現鈔買入"], df["現鈔賣出"], df["即期買入"], df["即期賣出"]], axis=1)
          load_df = pd.concat([load_df, df["現鈔買入"]], axis=1)
          filename = filename.replace(".csv", "")
          name_list.extend([f"{filename}現鈔買入"])
  load_df.columns = name_list

  # 反向排序
  load_df = load_df.iloc[::-1]

  # 排序column
  load_df = load_df.reindex(sorted(load_df.columns), axis=1)

  # 處理空值
  load_df.replace("-", 0, inplace=True)

  return load_df

#Kaggle 原路徑
# test_df = load_test_data("/kaggle/input/2023-artificial-intelligence-hw1/Dataset/test")

test_df = load_test_data("./Dataset/test")

test_df

In [ ]:
test = test_df.to_numpy()
test_size, feature_size = test.shape
# 因為 test 資料已經事先切割好範圍，故需要明確切分每段資料
test_size = test_size//input_date_data_size
test_x = np.empty([test_size, feature_size * input_date_data_size], dtype = float)

for idx in range(test_size):
  temp_data = np.array([])
  for count in range(input_date_data_size):
    temp_data = np.hstack([temp_data, test[idx * input_date_data_size + count]])
  test_x[idx, :] = temp_data

# row_sums = test_x.sum(axis=1)
# new_matrix = test_x / row_sums[:, np.newaxis]

# test 資料也需要照 training 方式做正規化
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]

# test_x = np.nan_to_num(test_x, nan=0)

# min_val = np.min(test_x, axis=0)
# max_val = np.max(test_x, axis=0)
# print(test_x[0],min_val,max_val)

# for i in range(len(test_x)):
#     for j in range(len(test_x[0])):
#         if max_val[j] - min_val[j] != 0:
#             test_x[i][j] = (test_x[i][j] - min_val[j]) / (max_val[j] - min_val[j])
#         else:
#             test_x[i][j] = (test_x[i][j] - min_val[j])

In [ ]:
test_x = torch.from_numpy(test_x.astype(np.float32))
predicted = model(test_x)

ids = [x for x in range(len(predicted))]
output_df = pd.DataFrame({'id': ids})
# 要按照規定順序設定col
currency_columns = ["AUD", "CAD", "EUR", "GBP", "HKD", "JPY", "KRW", "USD"]

for i, column_name in enumerate(currency_columns):
    output_df[column_name] = [x[i] for x in predicted.tolist()]


output_df.to_csv('/kaggle/working/submission.csv', index=False)